In [19]:
import matplotlib.pyplot as plt
from funcs import *
import os

os.chdir(R'C:\Users\adria\Desktop\STUDIA_FOLDERY\TELEDETEKCJA')

In [20]:
cir_raster = r"TIFs\CIR_2015.tif"
cir_dataset = read_spatial_raster(cir_raster)

rgb_raster = r"TIFs\RGB_2015.tif"
rgb_dataset = read_spatial_raster(rgb_raster)

band_nir = cir_dataset.GetRasterBand(1)
array_nir = band_nir.ReadAsArray()
array_nir = np.copy(array_nir)

band_red = cir_dataset.GetRasterBand(2)
array_red = band_red.ReadAsArray()
array_red = np.copy(array_red)

band_green = cir_dataset.GetRasterBand(3)
array_green = band_green.ReadAsArray()
array_green = np.copy(array_green)

band_blue = rgb_dataset.GetRasterBand(3)
array_blue = band_blue.ReadAsArray()
array_blue = np.copy(array_blue)

c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: CIR_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)
c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: RGB_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)


In [21]:
ids = np.array([7, 0, 3])

vector_features = gpd.read_file(R"budynki\budynki.shp")

features = reproject_geodataframe(vector_features, cir_dataset.GetProjection())
features = convert_to_pixel_system(features, cir_dataset.GetGeoTransform())

## kanały

In [22]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  
    
    print(f"Obiekt {id}")
    
    # nir
    masked_fragment = np.copy(fragment_nir)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_nir[~no_data_mask]  # Select just valid pixels
    print(f"NIR: min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\nir_budynek{id}.png", masked_fragment, cmap='gray')

    # red
    masked_fragment = np.copy(fragment_red)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_red[~no_data_mask]
    print(f"Red min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\red_budynek{id}.png", masked_fragment, cmap='gray')

    # green
    masked_fragment = np.copy(fragment_green)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_green[~no_data_mask]
    print(f"Green min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\green_budynek{id}.png", masked_fragment, cmap='gray')

    # blue
    masked_fragment = np.copy(fragment_blue)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_blue[~no_data_mask] 
    print(f"Blue min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\blue_budynek{id}.png", masked_fragment, cmap='gray')

Obiekt 7
NIR: min: 48, max: 157, mean: 77.2828999211978
Red min: 40, max: 131, mean: 70.44365642237983
Green min: 43, max: 134, mean: 71.22695035460993
Blue min: 37, max: 120, mean: 65.39322301024428
Obiekt 0
NIR: min: 39, max: 187, mean: 101.36163265306122
Red min: 47, max: 203, mean: 127.01306122448979
Green min: 60, max: 217, mean: 132.78448979591838
Blue min: 43, max: 205, mean: 128.09142857142857
Obiekt 3
NIR: min: 59, max: 178, mean: 90.41953488372093
Red min: 60, max: 189, mean: 98.90325581395349
Green min: 55, max: 193, mean: 96.62139534883721
Blue min: 44, max: 191, mean: 92.05953488372093


## kompozycje

In [23]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    # rgb
    rgb = np.dstack([fragment_red, fragment_green, fragment_blue])
    # rgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgb_budynek{id}.png", rgb, cmap='gray')

    # irgb
    irgb = np.dstack([fragment_nir, fragment_green, fragment_blue])
    # irgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irgb_budynek{id}.png", irgb, cmap='gray')

    # irrb
    irrb = np.dstack([fragment_nir, fragment_red, fragment_blue])
    # irrb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irrb_budynek{id}.png", irrb, cmap='gray')

    # rgir
    rgir = np.dstack([fragment_red, fragment_green, fragment_nir])
    # rgir[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgir_budynek{id}.png", rgir, cmap='gray')

## wskaźniki 2015

In [24]:
print('2015\n')
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    print(f"Obiekt {id}")

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
    # ndvi[no_data_mask] = 0
    ndvi[np.isnan(ndvi)] = 0
    ndvi = ndvi/1 * 255
    ndvi = np.uint8(ndvi)
    print(f"NDVI min: {ndvi.min()}, max: {ndvi.max()}, mean: {ndvi.mean()}")
    
    plt.imsave(f"tex\\spektralne\\ndvi_budynek{id}_2015.png", ndvi, cmap='gray')

    bi = (fragment_red - fragment_nir) / (fragment_red + fragment_nir)
    bi[np.isnan(bi)] = 0
    bi = bi/1 * 255
    bi = np.uint8(bi)
    print(f"BI min: {bi.min()}, max: {bi.max()}, mean: {bi.mean()}\n")

    plt.imsave(f"tex\\spektralne\\bi_budynek{id}_2015.png", bi, cmap='gray')

2015

Obiekt 7
NDVI min: 0, max: 255, mean: 54.66478129713424
BI min: 0, max: 255, mean: 108.24509803921569

Obiekt 0
NDVI min: 0, max: 255, mean: 93.33318425760287
BI min: 0, max: 255, mean: 73.35420393559929

Obiekt 3
NDVI min: 0, max: 255, mean: 66.43285714285715
BI min: 0, max: 255, mean: 84.71523809523809



C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:44: RuntimeWarning: divide by zero encountered in divide
  ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:48: RuntimeWarning: invalid value encountered in cast
  ndvi = np.uint8(ndvi)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:53: RuntimeWarning: divide by zero encountered in divide
  bi = (fragment_red - fragment_nir) / (fragment_red + fragment_nir)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:56: RuntimeWarning: invalid value encountered in cast
  bi = np.uint8(bi)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:44: RuntimeWarning: divide by zero encountered in divide
  ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\2306746632.py:48: RuntimeWarning: invalid value encountered in cast
  ndvi = np.uint8(ndvi)
C:\U

## rastry 2023

In [25]:
cir_raster = r"TIFs\CIR_2023.tif"
cir_dataset = read_spatial_raster(cir_raster)

rgb_raster = r"TIFs\RGB_2023.tif"
rgb_dataset = read_spatial_raster(rgb_raster)

band_nir = cir_dataset.GetRasterBand(1)
array_nir = band_nir.ReadAsArray()
array_nir = np.copy(array_nir)

band_red = cir_dataset.GetRasterBand(2)
array_red = band_red.ReadAsArray()
array_red = np.copy(array_red)

band_green = cir_dataset.GetRasterBand(3)
array_green = band_green.ReadAsArray()
array_green = np.copy(array_green)

band_blue = rgb_dataset.GetRasterBand(3)
array_blue = band_blue.ReadAsArray()
array_blue = np.copy(array_blue)

ids = np.array([7, 0, 3])

vector_features = gpd.read_file(R"budynki\budynki.shp")

features = reproject_geodataframe(vector_features, cir_dataset.GetProjection())
features = convert_to_pixel_system(features, cir_dataset.GetGeoTransform())

## wskaźniki 2023

In [26]:
print('2023\n')
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    print(f"Obiekt {id}")

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
    # ndvi[no_data_mask] = 0
    ndvi[np.isnan(ndvi)] = 0
    ndvi = ndvi/1 * 255
    ndvi = np.uint8(ndvi)
    print(f"NDVI min: {ndvi.min()}, max: {ndvi.max()}, mean: {ndvi.mean()}")
    
    plt.imsave(f"tex\\spektralne\\ndvi_budynek{id}_2023.png", ndvi, cmap='gray')

    bi = (fragment_red - fragment_nir) / (fragment_red + fragment_nir)
    bi[np.isnan(bi)] = 0
    bi = bi/1 * 255
    bi = np.uint8(bi)
    print(f"BI min: {bi.min()}, max: {bi.max()}, mean: {bi.mean()}\n")

    plt.imsave(f"tex\\spektralne\\bi_budynek{id}_2023.png", bi, cmap='gray')

2023

Obiekt 7
NDVI min: 0, max: 255, mean: 107.98076923076923
BI min: 0, max: 255, mean: 81.65987933634993

Obiekt 0
NDVI min: 0, max: 255, mean: 104.83944543828265
BI min: 0, max: 255, mean: 91.61001788908766

Obiekt 3
NDVI min: 0, max: 255, mean: 99.61
BI min: 0, max: 255, mean: 93.04380952380953



C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:44: RuntimeWarning: divide by zero encountered in divide
  ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:48: RuntimeWarning: invalid value encountered in cast
  ndvi = np.uint8(ndvi)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:53: RuntimeWarning: divide by zero encountered in divide
  bi = (fragment_red - fragment_nir) / (fragment_red + fragment_nir)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:56: RuntimeWarning: invalid value encountered in cast
  bi = np.uint8(bi)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:44: RuntimeWarning: divide by zero encountered in divide
  ndvi = (fragment_nir - fragment_red) / (fragment_nir + fragment_red)
C:\Users\adria\AppData\Local\Temp\ipykernel_27440\3671387836.py:48: RuntimeWarning: invalid value encountered in cast
  ndvi = np.uint8(ndvi)
C:\U